</div>
<div align="center">
  <img src="img/evolnode.png" width="800" alt="Fourier reconstruction convergence">
  <p><em> Evolution Node free you from coding and debugging, let LLM evolve your code for you.</em></p>
</div>
<div align="left">
<p><em>Bored of manual coding, a function? EvolNode let LLM automate function design and guide the evolution of it with genetic algorithm. A node here takes a task, input, and output, it uses either code or another LLM to complete the task, ensuring aligned input and output value types and ,.names.Fitness evaluation is done by running the function with a few specified test cases, the more diverse the test case, the better the evolution.
</em></p>
</div>

In [1]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode


# Code + Compilor Task
# mp = MetaPrompt("Generate Fibonacci sequence.", "fibonacci", ["n"], ["sequence"], ["int"], ["list"], PromptMode.CODE) # 
# node = EvolNode(mp, None, None)
# input_dict = {"n": 10}
# reasoning, code = node.evolve([input_dict], "i1", replace=True) # Evolution with guaranteed structural fitness
# node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 
node = EvolNode(mp, None, None)
input_dict = {"name": "Dilireba"}
reasoning, code = node.evolve([input_dict], "i1", replace=True) # evolution node assume access to 'test_cases', evalnode should contain those
output_dict = node(input_dict) # LLM-is stochastic, so chances of mal-functioning here ... (Aim to resolve with massive inference)

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
# node._extend_test_cases(10) # tested fine 

node.get_test_cases(100)

Generating test cases:  30%|███       | 30/100 [00:21<00:51,  1.35case/s]

In [9]:
r = node.temp_response

print(r)

# from methods.meta_prompt import extract_json_from_text

# # extract_json_from_text(r)

```json
[
    {
        'input': {'name': 'Taylor Swift'},
        'expected_output': {'age': 33}
    },
    {
        'input': {'name': 'Elvis Presley'},
        'expected_output': {'age': 88}  // Considering hypothetical current age if still alive
    },
    {
        'input': {'name': 'Zendaya'},
        'expected_output': {'age': 27}
    },
    {
        'input': {'name': 'Albert Einstein'},
        'expected_output': {'age': 144}  // Considering hypothetical current age if still alive
    },
    {
        'input': {'name': 'Keanu Reeves'},
        'expected_output': {'age': 59}
    },
    {
        'input': {'name': 'Marilyn Monroe'},
        'expected_output': {'age': 97}  // Considering hypothetical current age if still alive
    },
    {
        'input': {'name': 'Michael Jordan'},
        'expected_output': {'age': 60}
    },
    {
        'input': {'name': 'Lady Gaga'},
        'expected_output': {'age': 37}
    },
    {
        'input': {'name': 'Queen Elizabeth II'},
      

In [2]:
# Generate Evaluation TestCases
from methods.llm import get_openai_response 
from methods.meta_prompt import extract_json_from_text 

eval_prompt = mp._get_eval_prompt()
response = get_openai_response(eval_prompt)
test_case_list = extract_json_from_text(response)
node.test_cases.extend([(d['input'], d['expected_output']) for d in test_case_list]) # bet it's a list

# Slot eval_dict into EvolNode.test_cases

In [7]:
test_case_list = extract_json_from_text(response)

node.test_cases.extend([(d['input'], d['expected_output']) for d in test_case_list]) # bet it's a list

In [8]:
node.test_cases

[({'name': 'Taylor Swift'}, {'age': 33}),
 ({'name': 'Will Smith'}, {'age': 55}),
 ({'name': 'Marilyn Monroe'}, {'age': 97}),
 ({'name': 'Jaden Smith'}, {'age': 25}),
 ({'name': 'Queen Elizabeth II'}, {'age': 97})]


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [1]:
from methods.meta_prompt import MetaPlan, extract_json_from_text
from methods.llm import get_openai_response
from methods.diagram import visualize_plan_dict

# Breaking Down EvolGraph.generate() function components here 

# Step 1: Generate Plan Dict
mp = MetaPlan("Help me date Dilireba, I'm a white colar worker living on salary.")
prompt = mp._get_prompt_i1()
response = get_openai_response(prompt)
plan_dict = extract_json_from_text(response)
# visualize_plan_dict(plan_dict)

Could not load vllm class, check CUDA support and GPU RAM size


Evaluation Node is critical here

In [2]:
# Eval Node 
get_response = get_openai_response 


prompt_content = mp._get_eval_prompt_i1()
response = get_response(prompt_content)
eval_dict = extract_json_from_text(response)


# eval_prompt = MetaPrompt(
#     task=node.get("task"),
#     func_name=node.get("name"),
#     inputs=node.get("input"),
#     outputs=node.get("output"),
#     mode=node.get("mode").lower()
# )
# eval_node = EvolNode(meta_prompt=eval_prompt)

In [3]:
print(prompt_content)

Goal: Evaluating wether the goal Help me date Dilireba, I'm a white colar worker living on salary. has been achieved.
First, describe the intuition for your tactics and main steps in one sentence. The description must be inside a brace.
Generate a JSON response describing a task to evaluate whehter the goal is completed.

**Output Format:**

Provide the output in the following JSON structure:
```json
{
    "task": "Evaluate Task",
    "name": "eval_goal",
    "input": "Inputs required for evaluation",
    "output": "Outputs required for evaluation",
    "target": "Purpose of the evaluation",
    "mode: "CODE"
}



In [ ]:
nodes = {}
for node in plan_dict["nodes"]:
    node_prompt = MetaPrompt(
        task=node.get("task"),
        func_name=node.get("name"),
        inputs=node.get("inputs"),
        outputs=node.get("outputs"),
        input_types=node.get("input_types"),
        output_types=node.get("output_types"),
        mode=node.get("mode").lower()
    )
    nodes[node.get("name")] = EvolNode(meta_prompt=node_prompt)

edges = plan_dict["edges"]

In [4]:
from methods.eoh_evolution import EvolGraph

eg = EvolGraph.read_from_dict(plan_dict)
# eg.evolve()

In [7]:
eg.generate()

TypeError: EvolGraph.generate() missing 1 required positional argument: 'goal'